# Multi-Layer Perceptron

A multi-layer perceptron (MLP) is a type of artificial neural network consisting of multiple layers of neurons. The neurons in the MLP typically use nonlinear activation functions, allowing the network to learn complex patterns in data. MLPs are significant in machine learning because they can learn nonlinear relationships in data, making them powerful models for tasks such as classification, regression, and pattern recognition.

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from torch.utils.data import DataLoader, TensorDataset


url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv'
data = pd.read_csv(url, sep=';')

X = data.drop('quality', axis=1).values
y = data['quality'].values

y = np.where(y >= 6, 1, 0)  # 1 - good one , 0 - bad one

scaler = StandardScaler()
X = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# transforming to tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).unsqueeze(1)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).unsqueeze(1)

# data-loader creation for mini-batches
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


class WineQualityMLP(nn.Module):
    def __init__(self):
        super(WineQualityMLP, self).__init__()
        self.fc1 = nn.Linear(11, 64)  # input layer (11 features → 64 neurons)
        self.fc2 = nn.Linear(64, 32)  # hidden layer (64 → 32)
        self.fc3 = nn.Linear(32, 1)   # output layer (32 → 1)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()   # sigmoid due to binarity

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.sigmoid(self.fc3(x))
        return x


model = WineQualityMLP()

# loss function and optimizer
criterion = nn.BCELoss()  # binary cross-entropy
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 60
for epoch in range(num_epochs):
    model.train()  # switch to training mode
    total_loss = 0
    
    for X_batch, y_batch in train_loader:
        optimizer.zero_grad()  # Обнуляем градиенты
        outputs = model(X_batch)  # forward porpagation
        loss = criterion(outputs, y_batch)  # calculate loss
        loss.backward()  # error backpropagation
        optimizer.step()  # new weights
        
        total_loss += loss.item()
    
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {total_loss / len(train_loader):.4f}')

model.eval()  # evaluation mode
correct = 0
total = 0

with torch.no_grad():
    for X_batch, y_batch in test_loader:
        outputs = model(X_batch)
        predicted = (outputs > 0.5).float()  # classification
        correct += (predicted == y_batch).sum().item()
        total += y_batch.size(0)

accuracy = correct / total
print(f'Accuracy on test data: {accuracy:.4f}')


Epoch [1/60], Loss: 0.6685
Epoch [2/60], Loss: 0.5884
Epoch [3/60], Loss: 0.5270
Epoch [4/60], Loss: 0.5100
Epoch [5/60], Loss: 0.5053
Epoch [6/60], Loss: 0.4985
Epoch [7/60], Loss: 0.4950
Epoch [8/60], Loss: 0.4900
Epoch [9/60], Loss: 0.4877
Epoch [10/60], Loss: 0.4817
Epoch [11/60], Loss: 0.4805
Epoch [12/60], Loss: 0.4763
Epoch [13/60], Loss: 0.4739
Epoch [14/60], Loss: 0.4718
Epoch [15/60], Loss: 0.4695
Epoch [16/60], Loss: 0.4654
Epoch [17/60], Loss: 0.4639
Epoch [18/60], Loss: 0.4641
Epoch [19/60], Loss: 0.4600
Epoch [20/60], Loss: 0.4591
Epoch [21/60], Loss: 0.4551
Epoch [22/60], Loss: 0.4538
Epoch [23/60], Loss: 0.4493
Epoch [24/60], Loss: 0.4488
Epoch [25/60], Loss: 0.4468
Epoch [26/60], Loss: 0.4459
Epoch [27/60], Loss: 0.4407
Epoch [28/60], Loss: 0.4427
Epoch [29/60], Loss: 0.4378
Epoch [30/60], Loss: 0.4341
Epoch [31/60], Loss: 0.4316
Epoch [32/60], Loss: 0.4311
Epoch [33/60], Loss: 0.4298
Epoch [34/60], Loss: 0.4269
Epoch [35/60], Loss: 0.4242
Epoch [36/60], Loss: 0.4202
E